<a href="https://colab.research.google.com/github/yelsinhc2025/PROGRANACION-102/blob/main/matriz_dispersa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Matriz Dispersa

## 1. Definición
Una **matriz dispersa** es una matriz en la que la mayoría de sus elementos son cero.  
En lugar de almacenar todos los elementos (incluyendo los ceros), se utilizan estructuras de datos que solo guardan los elementos distintos de cero junto con sus posiciones.

**Ejemplo de matriz densa (con muchos ceros):**

$$
A = \begin{bmatrix}
0 & 0 & 3 \\
0 & 0 & 0 \\
4 & 0 & 0 \\
\end{bmatrix}
$$

En este caso, almacenar todos los elementos es ineficiente. En cambio, usamos representaciones dispersas.


## 2. Representaciones dispersas de A

Usaremos **indexado base 0** (estilo SciPy) para las coordenadas `(fila, columna)`.

Matriz densa:
$$
A=\begin{bmatrix}
0 & 0 & 3\\
0 & 0 & 0\\
4 & 0 & 0
\end{bmatrix}
$$
Elementos ≠ 0: \((0,2)=3\) y \((2,0)=4\).

---

## (1) COO — *Coordinate list*
Guarda listas paralelas `row`, `col`, `data` para cada elemento no-cero.

- `row = [0, 2]`  
- `col = [2, 0]`  
- `data = [3, 4]`  
- `shape = (3, 3)`

---

## (2) DOK — *Dictionary of Keys*
Diccionario cuyas claves son coordenadas y valores son los no-ceros.

- `data = { (0, 2): 3, (2, 0): 4 }`  
- `shape = (3, 3)`

---

## (3) LIL — *List of Lists*
Para cada fila, una lista de pares `(columna, valor)`.

- Fila 0 → `[(2, 3)]`  
- Fila 1 → `[]`  
- Fila 2 → `[(0, 4)]`  
- `shape = (3, 3)`

(Equivalente como estructura: `rows = [[(2,3)], [], [(0,4)]]`)

---

## (4) CSR — *Compressed Sparse Row*
Tres arreglos: `data` (valores), `indices` (columnas), `indptr` (punteros por fila).

- Recorrido por filas (columnas ordenadas dentro de cada fila):
  - Fila 0: col `2` → valor `3`  
  - Fila 1: —  
  - Fila 2: col `0` → valor `4`

Arreglos:
- `data   = [3, 4]`            ← valores no-cero en orden por filas  
- `indices= [2, 0]`            ← columnas correspondientes  
- `indptr = [0, 1, 1, 2]`      ← inicios de fila en `data/indices`  
  - (Fila 0: `data[0:1]`, Fila 1: `data[1:1]` vacío, Fila 2: `data[1:2]`)  
- `shape  = (3, 3)`


## 3. Usando Scipy

### 3.1. COO — Coordinate list

In [ ]:
from scipy.sparse import coo_matrix
import numpy as np

A_dense = np.array([
    [0, 0, 3],
    [0, 0, 0],
    [4, 0, 0]
])

A_coo = coo_matrix(A_dense)
print("data:", A_coo.data)     # [3 4]
print("row:", A_coo.row)       # [0 2]
print("col:", A_coo.col)       # [2 0]

data: [3 4]
row: [0 2]
col: [2 0]


### 3.2. DOK — Dictionary of Keys

In [ ]:
from scipy.sparse import dok_matrix

A_dok = dok_matrix(A_dense)
print(dict(A_dok))  # {(0, 2): 3, (2, 0): 4}


{(np.int32(0), np.int32(2)): np.int64(3), (np.int32(2), np.int32(0)): np.int64(4)}


### 3.3. LIL — List of Lists

In [ ]:
from scipy.sparse import lil_matrix

A_lil = lil_matrix(A_dense)
print(A_lil.rows)  # [[2], [], [0]]
print(A_lil.data)  # [[3], [], [4]]


[list([2]) list([]) list([0])]
[list([3]) list([]) list([4])]


### 3.4. CSR — Compressed Sparse Row

In [ ]:
from scipy.sparse import csr_matrix

A_csr = csr_matrix(A_dense)
print("data:   ", A_csr.data)     # [3 4]
print("indices:", A_csr.indices)  # [2 0]
print("indptr: ", A_csr.indptr)   # [0 1 1 2]


data:    [3 4]
indices: [2 0]
indptr:  [0 1 1 2]


# Comparación de formatos de matrices dispersas en SciPy

| Formato | Estructura interna | Ventajas principales | Desventajas | Casos de uso típicos |
|---------|-------------------|----------------------|-------------|-----------------------|
| **COO** (Coordinate List) | Tres arrays: `row`, `col`, `data` | Muy simple, fácil de construir; eficiente para ensamblaje inicial | No eficiente para operaciones repetitivas (suma, multiplicación); no permite indexado rápido | Construcción inicial de matrices dispersas antes de convertir a otro formato |
| **DOK** (Dictionary of Keys) | Diccionario: `(fila, col) → valor` | Fácil de modificar e insertar elementos; intuitivo | Operaciones algebraicas lentas; ocupa más memoria por overhead de diccionario | Construcción incremental, edición de elementos individuales |
| **LIL** (List of Lists) | Cada fila tiene listas de `col` y `data` | Fácil de modificar por filas; eficiente en asignación de filas completas | No eficiente para operaciones matriciales globales | Construcción de matrices fila por fila antes de conversión a CSR/CSC |
| **CSR** (Compressed Sparse Row) | Tres arrays: `data`, `indices`, `indptr` | Muy eficiente para operaciones algebraicas (multiplicar por vector, slicing por filas) | Difícil de modificar (inserciones son costosas) | Computación numérica, Machine Learning, álgebra lineal dispersa |


## 4. Comparación de tamaños



In [ ]:
import numpy as np
from scipy.sparse import csr_matrix

# Creamos una matriz grande 1000x1000 con pocos valores distintos de cero
np.random.seed(42)
A_dense = np.zeros((1000, 1000), dtype=np.float32)

# Insertamos solo 1% de valores distintos de cero
num_nonzeros = int(0.01 * A_dense.size)
rows = np.random.randint(0, A_dense.shape[0], num_nonzeros)
cols = np.random.randint(0, A_dense.shape[1], num_nonzeros)
A_dense[rows, cols] = np.random.rand(num_nonzeros)

# Convertir a formato disperso CSR
A_sparse = csr_matrix(A_dense)


In [ ]:
import sys

# Función auxiliar para calcular memoria aproximada
def get_size(obj, name="obj"):
    size = sys.getsizeof(obj)
    print(f"{name} ocupa aproximadamente {size/1024:.2f} KB")

print("Tamaño aproximado de objetos:")
get_size(A_dense, "Matriz densa (NumPy)")
get_size(A_sparse, "Matriz dispersa (CSR)")


Tamaño aproximado de objetos:
Matriz densa (NumPy) ocupa aproximadamente 3906.38 KB
Matriz dispersa (CSR) ocupa aproximadamente 0.05 KB


## 5. Bag-of-Words para clasificación de textos

Corpus de documentos:

In [ ]:
docs = [
    "oferta para el nuevo iPhone", #spam
    "nueva tarea del curso de Algoritmos y Estructuras de Datos", #no
    "visita la biblioteca de la universidad",#no
    "no te pierdas la oferta de viaje a Europa", #spam
    "anuncio de la universidad del curso de Algoritmos y Estructuras de Datos",#no
    "nueva oferta para el curso de algoritmos para fotografía y datos", #spam
]
y = [1,0,0,1,0,1]  # etiquetas (ejemplo)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

pipe = Pipeline([
    ("bow", CountVectorizer(lowercase=True, analyzer="word")),  # genera una CSR
    ("clf", MultinomialNB()),                                   # usa directamente la matriz dispersa
])

X_train, X_test, y_train, y_test = train_test_split(docs, y, test_size=0.2, random_state=42, stratify=y)
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



In [ ]:
X_train

['visita la biblioteca de la universidad',
 'nueva oferta para el curso de algoritmos para fotografía y datos',
 'nueva tarea del curso de Algoritmos y Estructuras de Datos',
 'no te pierdas la oferta de viaje a Europa']

In [ ]:
vectorizer=CountVectorizer(lowercase=True, analyzer="word")
x = vectorizer.fit_transform(X_train)
df = pd.DataFrame(
    x.toarray(),
    columns=vectorizer.get_feature_names_out()
)

print(df)

   algoritmos  biblioteca  curso  datos  de  del  el  estructuras  europa  \
0           0           1      0      0   1    0   0            0       0   
1           1           0      1      1   1    0   1            0       0   
2           1           0      1      1   2    1   0            1       0   
3           0           0      0      0   1    0   0            0       1   

   fotografía  ...  no  nueva  oferta  para  pierdas  tarea  te  universidad  \
0           0  ...   0      0       0     0        0      0   0            1   
1           1  ...   0      1       1     2        0      0   0            0   
2           0  ...   0      1       0     0        0      1   0            0   
3           0  ...   1      0       1     0        1      0   1            0   

   viaje  visita  
0      0       1  
1      0       0  
2      0       0  
3      1       0  

[4 rows x 21 columns]


In [ ]:
X_train

['visita la biblioteca de la universidad',
 'nueva oferta para el curso de algoritmos para fotografía y datos',
 'nueva tarea del curso de Algoritmos y Estructuras de Datos',
 'no te pierdas la oferta de viaje a Europa']

## 6. Modelos de Grafos Causales para Machine Learning

In [ ]:
!pip install inteligenciartificial

In [ ]:
from inteligenciartificial.modelografosprobabilisticos import NaiveBayes
from inteligenciartificial.modelografosprobabilisticos import RedBayesiana
from inteligenciartificial.modelografosprobabilisticos.busqueda import K2
from inteligenciartificial.modelografosprobabilisticos import Grafo
from inteligenciartificial.modelografosprobabilisticos import DistribucionDiscreta
from inteligenciartificial.modelografosprobabilisticos.metricas import Entropia, BIC
from inteligenciartificial.modelografosprobabilisticos.utils import accuracy_score
import pandas as pd

# Lectura de dataset
df_train = pd.read_csv("weather.nominal_train.csv")
df_test = pd.read_csv("weather.nominal_test.csv")

y_train = df_train["play"]
X_train = df_train.drop(columns=["play"])

y_test = df_test["play"]
X_test = df_test.drop(columns=["play"])

# Agregamos las aristas a nuestro grafo
g = Grafo([("outlook", "play"), ("windy", "play"), ("temperature", "play"), ("humidity", "play")])

# Creamos otra red bayesiana ad-hoc
rb2 = RedBayesiana(g)
rb2.fit(X_train, y_train) #Entrenamiento
dPlay2 = rb2.getDistribucion("play")
print("Distribucion de Play es: ", dPlay2)
dWindy2 = rb2.getDistribucion("windy")
print("Distribucion de Windy es: ", dWindy2)

#Predicción
y_pred2 = rb2.predict(X_test)
acc2 = accuracy_score(y_test, y_pred2)
print("Accuracy de RB2 es: ", acc2)


Distribucion de Play es:  P(play | humidity,outlook,temperature,windy) con 72 entradas
Distribucion de Windy es:  P(windy)={False: np.float64(0.5), True: np.float64(0.5)}
Accuracy de RB2 es:  0.5
